# Exploratory Data Analysis (EDA) - S&OP Dataset

**Sales & Operations Planning - Comprehensive Data Exploration**

Author: Aviwe Dlepu
Date: October 2025

## Objective
Conduct thorough exploratory data analysis on S&OP datasets to:
- Understand data structure and quality
- Identify patterns and relationships
- Detect anomalies and outliers
- Generate actionable insights
- Inform modeling strategies

## 1. Environment Setup & Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import missingno as msno

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# AD Solutions color scheme
AD_COLORS = {
    'primary': '#CC5500',
    'secondary': '#B8470B',
    'accent': '#FF6347',
    'highlight': '#FFA500',
    'neutral': '#8B4513'
}

print("🔍 EDA Environment Setup Complete")
print("Project: S&OP Exploratory Data Analysis")
print("Author: Aviwe Dlepu")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("=" * 60)

In [ ]:
# Load datasets with error handling
datasets = {}
file_mapping = {
    'orders': 'final_orders_perfect_integrity.csv',
    'products': 'final_products_perfect_integrity.csv',
    'inventory': 'final_inventory_perfect_integrity.csv',
    'forecasts': 'final_forecasts_perfect_integrity.csv'
}

print("📂 Loading S&OP Datasets...")
for dataset_name, filename in file_mapping.items():
    try:
        df = pd.read_csv(filename)
        datasets[dataset_name] = df
        print(f"✅ {dataset_name.capitalize()}: {len(df):,} records, {len(df.columns)} columns")
    except FileNotFoundError:
        print(f"❌ File not found: {filename}")
    except Exception as e:
        print(f"❌ Error loading {dataset_name}: {e}")

print(f"\n📊 Total datasets loaded: {len(datasets)}")

# Quick overview
if datasets:
    total_records = sum(len(df) for df in datasets.values())
    print(f"📈 Total records across all datasets: {total_records:,}")
else:
    print("⚠️ No datasets loaded. Please check file paths and try again.")

## 2. Dataset Structure Analysis

In [ ]:
# Comprehensive dataset overview
def analyze_dataset_structure(df, dataset_name):
    print(f"\n{'='*20} {dataset_name.upper()} DATASET ANALYSIS {'='*20}")
    
    # Basic information
    print(f"📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"💾 Memory Usage: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    
    # Data types
    print(f"\n📋 Data Types:")
    dtype_counts = df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"   {dtype}: {count} columns")
    
    # Missing values
    missing_values = df.isnull().sum()
    missing_pct = (missing_values / len(df)) * 100
    missing_summary = pd.DataFrame({
        'Missing_Count': missing_values,
        'Missing_Percentage': missing_pct
    })
    missing_summary = missing_summary[missing_summary['Missing_Count'] > 0]
    
    if len(missing_summary) > 0:
        print(f"\n❌ Missing Values Found:")
        print(missing_summary.round(2))
    else:
        print(f"\n✅ No missing values detected")
    
    # Duplicate rows
    duplicates = df.duplicated().sum()
    print(f"\n🔄 Duplicate rows: {duplicates:,} ({(duplicates/len(df)*100):.2f}%)")
    
    # Unique values per column
    print(f"\n🔢 Unique Values per Column:")
    for col in df.columns[:10]:  # Show first 10 columns
        unique_count = df[col].nunique()
        unique_pct = (unique_count / len(df)) * 100
        print(f"   {col}: {unique_count:,} ({unique_pct:.1f}%)")
    
    if len(df.columns) > 10:
        print(f"   ... and {len(df.columns) - 10} more columns")
    
    return missing_summary

# Analyze each dataset
missing_data_summary = {}
for name, df in datasets.items():
    missing_summary = analyze_dataset_structure(df, name)
    missing_data_summary[name] = missing_summary

In [ ]:
# Data quality visualization
if datasets:
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))
    fig.suptitle('🔍 Data Quality Overview - S&OP Datasets', fontsize=16, fontweight='bold')
    
    dataset_names = list(datasets.keys())
    
    for i, (name, df) in enumerate(datasets.items()):
        if i < 4:  # Limit to 4 datasets
            row, col = i // 2, i % 2
            
            # Missing data visualization
            missing_counts = df.isnull().sum()
            if missing_counts.sum() > 0:
                missing_counts[missing_counts > 0].plot(kind='bar', ax=axes[row, col], 
                                                       color=AD_COLORS['primary'])
                axes[row, col].set_title(f'{name.capitalize()} - Missing Values')
                axes[row, col].set_ylabel('Missing Count')
                axes[row, col].tick_params(axis='x', rotation=45)
            else:
                axes[row, col].text(0.5, 0.5, f'{name.capitalize()}\n✅ No Missing Values', 
                                  ha='center', va='center', fontsize=12, fontweight='bold')
                axes[row, col].set_xlim(0, 1)
                axes[row, col].set_ylim(0, 1)
                axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

    # Dataset size comparison
    dataset_sizes = {name: len(df) for name, df in datasets.items()}
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    bars = ax.bar(dataset_sizes.keys(), dataset_sizes.values(), 
                  color=[AD_COLORS['primary'], AD_COLORS['secondary'], 
                        AD_COLORS['accent'], AD_COLORS['highlight']])
    
    ax.set_title('📊 Dataset Size Comparison', fontsize=14, fontweight='bold')
    ax.set_ylabel('Number of Records')
    ax.set_xlabel('Dataset')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{int(height):,}', ha='center', va='bottom', fontweight='bold')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 3. Orders Dataset Deep Dive

In [ ]:
if 'orders' in datasets:
    orders_df = datasets['orders'].copy()
    
    print("📦 ORDERS DATASET ANALYSIS")
    print("=" * 40)
    
    # Convert date column
    if 'order_date' in orders_df.columns:
        orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])
        print(f"📅 Date Range: {orders_df['order_date'].min()} to {orders_df['order_date'].max()}")
        print(f"📈 Time Span: {(orders_df['order_date'].max() - orders_df['order_date'].min()).days} days")
    
    # Revenue metrics
    if 'order_value' in orders_df.columns:
        total_revenue = orders_df['order_value'].sum()
        avg_order_value = orders_df['order_value'].mean()
        median_order_value = orders_df['order_value'].median()
        
        print(f"\n💰 Revenue Metrics:")
        print(f"   Total Revenue: R{total_revenue:,.2f}")
        print(f"   Average Order Value: R{avg_order_value:,.2f}")
        print(f"   Median Order Value: R{median_order_value:,.2f}")
        print(f"   Revenue Std Dev: R{orders_df['order_value'].std():,.2f}")
    
    # Customer segments
    if 'customer_type' in orders_df.columns:
        customer_dist = orders_df['customer_type'].value_counts()
        print(f"\n👥 Customer Type Distribution:")
        for customer_type, count in customer_dist.items():
            pct = (count / len(orders_df)) * 100
            print(f"   {customer_type}: {count:,} orders ({pct:.1f}%)")
    
    # Regional distribution
    if 'region' in orders_df.columns:
        regional_dist = orders_df['region'].value_counts()
        print(f"\n🌍 Regional Distribution:")
        for region, count in regional_dist.items():
            pct = (count / len(orders_df)) * 100
            print(f"   {region}: {count:,} orders ({pct:.1f}%)")
    
    # Product categories
    if 'category' in orders_df.columns:
        category_dist = orders_df['category'].value_counts()
        print(f"\n🏷️ Product Category Distribution:")
        for category, count in category_dist.head().items():
            pct = (count / len(orders_df)) * 100
            print(f"   {category}: {count:,} orders ({pct:.1f}%)")

else:
    print("❌ Orders dataset not available")

In [ ]:
# Orders data visualizations
if 'orders' in datasets:
    orders_df = datasets['orders'].copy()
    
    if 'order_date' in orders_df.columns:
        orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('📦 Orders Dataset - Exploratory Analysis', fontsize=16, fontweight='bold')
    
    # 1. Order value distribution
    if 'order_value' in orders_df.columns:
        axes[0, 0].hist(orders_df['order_value'], bins=50, color=AD_COLORS['primary'], alpha=0.7, edgecolor='black')
        axes[0, 0].set_title('Order Value Distribution')
        axes[0, 0].set_xlabel('Order Value (R)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].axvline(orders_df['order_value'].mean(), color='red', linestyle='--', label='Mean')
        axes[0, 0].axvline(orders_df['order_value'].median(), color='orange', linestyle='--', label='Median')
        axes[0, 0].legend()
    
    # 2. Customer type breakdown
    if 'customer_type' in orders_df.columns:
        customer_counts = orders_df['customer_type'].value_counts()
        axes[0, 1].pie(customer_counts.values, labels=customer_counts.index, autopct='%1.1f%%',
                      colors=[AD_COLORS['primary'], AD_COLORS['secondary'], AD_COLORS['accent']])
        axes[0, 1].set_title('Customer Type Distribution')
    
    # 3. Regional performance
    if 'region' in orders_df.columns and 'order_value' in orders_df.columns:
        regional_revenue = orders_df.groupby('region')['order_value'].sum().sort_values()
        axes[0, 2].barh(regional_revenue.index, regional_revenue.values, color=AD_COLORS['highlight'])
        axes[0, 2].set_title('Revenue by Region')
        axes[0, 2].set_xlabel('Total Revenue (R)')
    
    # 4. Monthly trend
    if 'order_date' in orders_df.columns and 'order_value' in orders_df.columns:
        monthly_revenue = orders_df.groupby(orders_df['order_date'].dt.to_period('M'))['order_value'].sum()
        axes[1, 0].plot(monthly_revenue.index.astype(str), monthly_revenue.values, 
                       marker='o', color=AD_COLORS['primary'], linewidth=3)
        axes[1, 0].set_title('Monthly Revenue Trend')
        axes[1, 0].set_ylabel('Revenue (R)')
        axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 5. Quantity distribution
    if 'qty' in orders_df.columns:
        axes[1, 1].hist(orders_df['qty'], bins=30, color=AD_COLORS['secondary'], alpha=0.7, edgecolor='black')
        axes[1, 1].set_title('Quantity Distribution')
        axes[1, 1].set_xlabel('Quantity')
        axes[1, 1].set_ylabel('Frequency')
    
    # 6. Discount analysis
    if 'discount_pct' in orders_df.columns:
        discount_data = orders_df[orders_df['discount_pct'] > 0]['discount_pct']
        if len(discount_data) > 0:
            axes[1, 2].hist(discount_data, bins=20, color=AD_COLORS['accent'], alpha=0.7, edgecolor='black')
            axes[1, 2].set_title('Discount Distribution (Non-Zero)')
            axes[1, 2].set_xlabel('Discount %')
            axes[1, 2].set_ylabel('Frequency')
        else:
            axes[1, 2].text(0.5, 0.5, 'No Discounts\nFound', ha='center', va='center')
            axes[1, 2].set_xlim(0, 1)
            axes[1, 2].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()

## 4. Statistical Analysis & Correlations

In [ ]:
# Statistical summary for numeric columns
if 'orders' in datasets:
    orders_df = datasets['orders']
    
    print("📊 STATISTICAL SUMMARY - ORDERS DATASET")
    print("=" * 50)
    
    # Numeric columns summary
    numeric_columns = orders_df.select_dtypes(include=[np.number]).columns
    print(f"\n🔢 Numeric Columns Analysis:")
    
    stats_summary = orders_df[numeric_columns].describe()
    print(stats_summary.round(2))
    
    # Correlation analysis
    if len(numeric_columns) > 1:
        print(f"\n🔗 Correlation Matrix:")
        correlation_matrix = orders_df[numeric_columns].corr()
        print(correlation_matrix.round(3))
        
        # Strong correlations
        print(f"\n💪 Strong Correlations (|r| > 0.7):")
        for i in range(len(correlation_matrix.columns)):
            for j in range(i+1, len(correlation_matrix.columns)):
                corr_value = correlation_matrix.iloc[i, j]
                if abs(corr_value) > 0.7:
                    var1 = correlation_matrix.columns[i]
                    var2 = correlation_matrix.columns[j]
                    print(f"   {var1} ↔ {var2}: {corr_value:.3f}")
    
    # Outlier detection for key metrics
    if 'order_value' in orders_df.columns:
        Q1 = orders_df['order_value'].quantile(0.25)
        Q3 = orders_df['order_value'].quantile(0.75)
        IQR = Q3 - Q1
        outlier_threshold_low = Q1 - 1.5 * IQR
        outlier_threshold_high = Q3 + 1.5 * IQR
        
        outliers = orders_df[(orders_df['order_value'] < outlier_threshold_low) | 
                           (orders_df['order_value'] > outlier_threshold_high)]
        
        print(f"\n🎯 Order Value Outliers:")
        print(f"   Total outliers: {len(outliers):,} ({len(outliers)/len(orders_df)*100:.1f}%)")
        print(f"   Lower threshold: R{outlier_threshold_low:,.2f}")
        print(f"   Upper threshold: R{outlier_threshold_high:,.2f}")
        
        if len(outliers) > 0:
            print(f"   Top 5 highest value orders:")
            top_orders = orders_df.nlargest(5, 'order_value')[['order_id', 'order_value', 'customer_type', 'region']]
            print(top_orders)

In [ ]:
# Correlation heatmap
if 'orders' in datasets:
    orders_df = datasets['orders']
    numeric_columns = orders_df.select_dtypes(include=[np.number]).columns
    
    if len(numeric_columns) > 1:
        # Create correlation heatmap
        plt.figure(figsize=(12, 8))
        correlation_matrix = orders_df[numeric_columns].corr()
        
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
                   square=True, linewidths=0.5, cbar_kws={"shrink": .8})
        
        plt.title('🔗 Orders Dataset - Correlation Matrix', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    # Box plots for categorical variables
    if 'customer_type' in orders_df.columns and 'order_value' in orders_df.columns:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # Order value by customer type
        orders_df.boxplot(column='order_value', by='customer_type', ax=axes[0])
        axes[0].set_title('Order Value by Customer Type')
        axes[0].set_xlabel('Customer Type')
        axes[0].set_ylabel('Order Value (R)')
        
        # Order value by region
        if 'region' in orders_df.columns:
            orders_df.boxplot(column='order_value', by='region', ax=axes[1])
            axes[1].set_title('Order Value by Region')
            axes[1].set_xlabel('Region')
            axes[1].set_ylabel('Order Value (R)')
            axes[1].tick_params(axis='x', rotation=45)
        
        plt.suptitle('📊 Order Value Distribution by Categories', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

## 5. Inventory Analysis

In [ ]:
if 'inventory' in datasets:
    inventory_df = datasets['inventory'].copy()
    
    print("📋 INVENTORY DATASET ANALYSIS")
    print("=" * 40)
    
    # Inventory value metrics
    if 'inventory_value' in inventory_df.columns:
        total_inventory = inventory_df['inventory_value'].sum()
        avg_inventory = inventory_df['inventory_value'].mean()
        median_inventory = inventory_df['inventory_value'].median()
        
        print(f"💰 Inventory Value Metrics:")
        print(f"   Total Inventory Value: R{total_inventory:,.2f}")
        print(f"   Average per Product: R{avg_inventory:,.2f}")
        print(f"   Median per Product: R{median_inventory:,.2f}")
    
    # Stock status distribution
    if 'stock_status' in inventory_df.columns:
        stock_dist = inventory_df['stock_status'].value_counts()
        print(f"\n📊 Stock Status Distribution:")
        for status, count in stock_dist.items():
            pct = (count / len(inventory_df)) * 100
            print(f"   {status}: {count:,} products ({pct:.1f}%)")
    
    # Warehouse distribution
    if 'warehouse' in inventory_df.columns:
        warehouse_dist = inventory_df['warehouse'].value_counts()
        print(f"\n🏢 Warehouse Distribution:")
        for warehouse, count in warehouse_dist.items():
            pct = (count / len(inventory_df)) * 100
            print(f"   {warehouse}: {count:,} products ({pct:.1f}%)")
    
    # High-value inventory items
    if 'inventory_value' in inventory_df.columns:
        top_inventory = inventory_df.nlargest(10, 'inventory_value')
        print(f"\n💎 Top 10 High-Value Inventory Items:")
        for _, row in top_inventory.iterrows():
            print(f"   {row['product_id']}: R{row['inventory_value']:,.2f} ({row.get('stock_status', 'N/A')})")

else:
    print("❌ Inventory dataset not available")

In [ ]:
# Inventory visualizations
if 'inventory' in datasets:
    inventory_df = datasets['inventory']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('📋 Inventory Dataset - Analysis Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Inventory value distribution
    if 'inventory_value' in inventory_df.columns:
        axes[0, 0].hist(inventory_df['inventory_value'], bins=50, color=AD_COLORS['primary'], 
                       alpha=0.7, edgecolor='black')
        axes[0, 0].set_title('Inventory Value Distribution')
        axes[0, 0].set_xlabel('Inventory Value (R)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].axvline(inventory_df['inventory_value'].mean(), color='red', 
                          linestyle='--', label='Mean')
        axes[0, 0].legend()
    
    # 2. Stock status pie chart
    if 'stock_status' in inventory_df.columns:
        stock_counts = inventory_df['stock_status'].value_counts()
        axes[0, 1].pie(stock_counts.values, labels=stock_counts.index, autopct='%1.1f%%',
                      colors=[AD_COLORS['primary'], AD_COLORS['secondary'], AD_COLORS['accent']])
        axes[0, 1].set_title('Stock Status Distribution')
    
    # 3. Available quantity distribution
    if 'available_qty' in inventory_df.columns:
        axes[1, 0].hist(inventory_df['available_qty'], bins=40, color=AD_COLORS['secondary'], 
                       alpha=0.7, edgecolor='black')
        axes[1, 0].set_title('Available Quantity Distribution')
        axes[1, 0].set_xlabel('Available Quantity')
        axes[1, 0].set_ylabel('Frequency')
    
    # 4. Inventory value by stock status
    if 'stock_status' in inventory_df.columns and 'inventory_value' in inventory_df.columns:
        status_value = inventory_df.groupby('stock_status')['inventory_value'].sum()
        axes[1, 1].bar(status_value.index, status_value.values, 
                      color=[AD_COLORS['primary'], AD_COLORS['accent'], AD_COLORS['highlight']])
        axes[1, 1].set_title('Total Inventory Value by Stock Status')
        axes[1, 1].set_xlabel('Stock Status')
        axes[1, 1].set_ylabel('Total Value (R)')
        axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## 6. Forecast Performance Analysis

In [ ]:
if 'forecasts' in datasets:
    forecasts_df = datasets['forecasts'].copy()
    
    print("🎯 FORECAST PERFORMANCE ANALYSIS")
    print("=" * 45)
    
    # Convert date column
    if 'forecast_month' in forecasts_df.columns:
        forecasts_df['forecast_month'] = pd.to_datetime(forecasts_df['forecast_month'])
    
    # Calculate absolute error
    if 'variance_pct' in forecasts_df.columns:
        forecasts_df['absolute_error'] = abs(forecasts_df['variance_pct'])
        forecasts_df['forecast_accuracy'] = 100 - forecasts_df['absolute_error']
    
    # Overall accuracy metrics
    if 'absolute_error' in forecasts_df.columns:
        overall_accuracy = 100 - forecasts_df['absolute_error'].mean()
        median_accuracy = 100 - forecasts_df['absolute_error'].median()
        
        print(f"📊 Overall Forecast Performance:")
        print(f"   Mean Accuracy: {overall_accuracy:.2f}%")
        print(f"   Median Accuracy: {median_accuracy:.2f}%")
        print(f"   Standard Deviation: {forecasts_df['absolute_error'].std():.2f}%")
    
    # Performance by forecast type
    if 'forecast_type' in forecasts_df.columns and 'absolute_error' in forecasts_df.columns:
        type_performance = forecasts_df.groupby('forecast_type').agg({
            'absolute_error': ['mean', 'median', 'std', 'count'],
            'forecast_accuracy': ['mean', 'median']
        }).round(2)
        
        print(f"\n🤖 Performance by Forecast Type:")
        print(type_performance)
    
    # Performance by confidence level
    if 'confidence_level' in forecasts_df.columns and 'absolute_error' in forecasts_df.columns:
        confidence_performance = forecasts_df.groupby('confidence_level').agg({
            'absolute_error': ['mean', 'count'],
            'forecast_accuracy': 'mean'
        }).round(2)
        
        print(f"\n📈 Performance by Confidence Level:")
        print(confidence_performance)
    
    # Best and worst forecasts
    if 'absolute_error' in forecasts_df.columns:
        best_forecasts = forecasts_df.nsmallest(5, 'absolute_error')
        worst_forecasts = forecasts_df.nlargest(5, 'absolute_error')
        
        print(f"\n🏆 Best 5 Forecasts (Lowest Error):")
        for _, row in best_forecasts.iterrows():
            print(f"   {row['product_id']}: {row['absolute_error']:.1f}% error ({row.get('forecast_type', 'N/A')})")
        
        print(f"\n⚠️ Worst 5 Forecasts (Highest Error):")
        for _, row in worst_forecasts.iterrows():
            print(f"   {row['product_id']}: {row['absolute_error']:.1f}% error ({row.get('forecast_type', 'N/A')})")

else:
    print("❌ Forecasts dataset not available")

In [ ]:
# Forecast performance visualizations
if 'forecasts' in datasets:
    forecasts_df = datasets['forecasts'].copy()
    
    if 'variance_pct' in forecasts_df.columns:
        forecasts_df['absolute_error'] = abs(forecasts_df['variance_pct'])
        forecasts_df['forecast_accuracy'] = 100 - forecasts_df['absolute_error']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🎯 Forecast Performance Analysis', fontsize=16, fontweight='bold')
    
    # 1. Forecast error distribution
    if 'absolute_error' in forecasts_df.columns:
        axes[0, 0].hist(forecasts_df['absolute_error'], bins=40, color=AD_COLORS['primary'], 
                       alpha=0.7, edgecolor='black')
        axes[0, 0].set_title('Forecast Error Distribution')
        axes[0, 0].set_xlabel('Absolute Error (%)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].axvline(forecasts_df['absolute_error'].mean(), color='red', 
                          linestyle='--', label=f"Mean: {forecasts_df['absolute_error'].mean():.1f}%")
        axes[0, 0].legend()
    
    # 2. Performance by forecast type
    if 'forecast_type' in forecasts_df.columns and 'forecast_accuracy' in forecasts_df.columns:
        type_accuracy = forecasts_df.groupby('forecast_type')['forecast_accuracy'].mean()
        axes[0, 1].bar(type_accuracy.index, type_accuracy.values, 
                      color=[AD_COLORS['primary'], AD_COLORS['secondary'], AD_COLORS['accent']])
        axes[0, 1].set_title('Average Accuracy by Forecast Type')
        axes[0, 1].set_xlabel('Forecast Type')
        axes[0, 1].set_ylabel('Accuracy (%)')
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Forecast vs Actual scatter plot
    if 'forecast_qty' in forecasts_df.columns and 'actual_demand' in forecasts_df.columns:
        sample_size = min(1000, len(forecasts_df))  # Sample for readability
        sample_df = forecasts_df.sample(sample_size)
        
        axes[1, 0].scatter(sample_df['forecast_qty'], sample_df['actual_demand'], 
                          alpha=0.6, color=AD_COLORS['primary'])
        
        # Perfect prediction line
        max_val = max(forecasts_df['forecast_qty'].max(), forecasts_df['actual_demand'].max())
        axes[1, 0].plot([0, max_val], [0, max_val], 'r--', label='Perfect Prediction')
        
        axes[1, 0].set_title('Forecast vs Actual Demand')
        axes[1, 0].set_xlabel('Forecast Quantity')
        axes[1, 0].set_ylabel('Actual Demand')
        axes[1, 0].legend()
    
    # 4. Performance by confidence level
    if 'confidence_level' in forecasts_df.columns and 'absolute_error' in forecasts_df.columns:
        forecasts_df.boxplot(column='absolute_error', by='confidence_level', ax=axes[1, 1])
        axes[1, 1].set_title('Error Distribution by Confidence Level')
        axes[1, 1].set_xlabel('Confidence Level')
        axes[1, 1].set_ylabel('Absolute Error (%)')
    
    plt.tight_layout()
    plt.show()

## 7. Cross-Dataset Analysis

In [ ]:
# Cross-dataset insights
print("🔗 CROSS-DATASET ANALYSIS")
print("=" * 35)

cross_insights = {}

# Orders vs Inventory analysis
if 'orders' in datasets and 'inventory' in datasets:
    orders_df = datasets['orders']
    inventory_df = datasets['inventory']
    
    # Product demand vs inventory levels
    product_demand = orders_df.groupby('product_id').agg({
        'qty': 'sum',
        'order_value': 'sum',
        'order_id': 'count'
    }).rename(columns={'qty': 'total_demand', 'order_value': 'total_revenue', 'order_id': 'order_count'})
    
    # Merge with inventory
    demand_inventory = product_demand.merge(
        inventory_df[['product_id', 'available_qty', 'inventory_value', 'stock_status']], 
        on='product_id', how='inner'
    )
    
    # Calculate inventory turnover
    demand_inventory['turnover_ratio'] = demand_inventory['total_revenue'] / demand_inventory['inventory_value']
    demand_inventory['demand_coverage'] = demand_inventory['available_qty'] / demand_inventory['total_demand']
    
    print(f"📊 Orders vs Inventory Insights:")
    print(f"   Products in both datasets: {len(demand_inventory):,}")
    print(f"   Average turnover ratio: {demand_inventory['turnover_ratio'].mean():.2f}")
    print(f"   Average demand coverage: {demand_inventory['demand_coverage'].mean():.2f}")
    
    # High turnover products
    high_turnover = demand_inventory.nlargest(5, 'turnover_ratio')
    print(f"\n🔥 Top 5 High Turnover Products:")
    for _, row in high_turnover.iterrows():
        print(f"   {row.name}: {row['turnover_ratio']:.2f} ratio ({row['stock_status']})")
    
    # Low turnover, high inventory
    low_turnover_high_inventory = demand_inventory[
        (demand_inventory['turnover_ratio'] < 1) & 
        (demand_inventory['inventory_value'] > demand_inventory['inventory_value'].quantile(0.75))
    ]
    
    print(f"\n⚠️ Low Turnover, High Inventory Products: {len(low_turnover_high_inventory):,}")
    if len(low_turnover_high_inventory) > 0:
        total_tied_up = low_turnover_high_inventory['inventory_value'].sum()
        print(f"   Total value tied up: R{total_tied_up:,.2f}")
    
    cross_insights['demand_inventory'] = demand_inventory

# Orders vs Forecasts analysis
if 'orders' in datasets and 'forecasts' in datasets:
    orders_df = datasets['orders']
    forecasts_df = datasets['forecasts']
    
    print(f"\n🎯 Orders vs Forecasts Analysis:")
    
    # Common products
    common_products = set(orders_df['product_id'].unique()) & set(forecasts_df['product_id'].unique())
    print(f"   Common products: {len(common_products):,}")
    print(f"   Orders-only products: {len(set(orders_df['product_id'].unique()) - common_products):,}")
    print(f"   Forecasts-only products: {len(set(forecasts_df['product_id'].unique()) - common_products):,}")

print(f"\n✅ Cross-dataset analysis completed!")
print(f"📈 Generated {len(cross_insights)} cross-dataset insights")

## 8. Time Series Analysis

In [ ]:
# Time series analysis for orders
if 'orders' in datasets:
    orders_df = datasets['orders'].copy()
    
    if 'order_date' in orders_df.columns:
        orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])
        
        print("📅 TIME SERIES ANALYSIS - ORDERS")
        print("=" * 40)
        
        # Daily trends
        daily_metrics = orders_df.groupby('order_date').agg({
            'order_value': ['sum', 'mean', 'count'],
            'qty': 'sum'
        }).round(2)
        
        daily_metrics.columns = ['daily_revenue', 'avg_order_value', 'order_count', 'total_qty']
        
        print(f"📊 Daily Metrics Summary:")
        print(f"   Date Range: {orders_df['order_date'].min()} to {orders_df['order_date'].max()}")
        print(f"   Total Days: {len(daily_metrics):,}")
        print(f"   Average Daily Revenue: R{daily_metrics['daily_revenue'].mean():,.2f}")
        print(f"   Average Daily Orders: {daily_metrics['order_count'].mean():.1f}")
        
        # Weekly patterns
        orders_df['day_of_week'] = orders_df['order_date'].dt.day_name()
        weekly_pattern = orders_df.groupby('day_of_week').agg({
            'order_value': ['sum', 'mean'],
            'order_id': 'count'
        }).round(2)
        
        weekly_pattern.columns = ['total_revenue', 'avg_order_value', 'order_count']
        
        # Reorder by weekday
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        weekly_pattern = weekly_pattern.reindex(day_order)
        
        print(f"\n📅 Weekly Pattern:")
        print(weekly_pattern)
        
        # Best and worst days
        best_day = weekly_pattern['total_revenue'].idxmax()
        worst_day = weekly_pattern['total_revenue'].idxmin()
        
        print(f"\n🏆 Best performing day: {best_day}")
        print(f"📉 Lowest performing day: {worst_day}")
        
        # Monthly trends
        monthly_trends = orders_df.groupby(orders_df['order_date'].dt.to_period('M')).agg({
            'order_value': 'sum',
            'order_id': 'count'
        }).round(2)
        
        monthly_trends.columns = ['monthly_revenue', 'monthly_orders']
        monthly_trends['growth_rate'] = monthly_trends['monthly_revenue'].pct_change() * 100
        
        print(f"\n📈 Monthly Growth Analysis:")
        print(monthly_trends)
        
        avg_growth = monthly_trends['growth_rate'].mean()
        print(f"\n📊 Average Monthly Growth Rate: {avg_growth:.2f}%")

else:
    print("❌ Orders dataset not available for time series analysis")

In [ ]:
# Time series visualizations
if 'orders' in datasets:
    orders_df = datasets['orders'].copy()
    
    if 'order_date' in orders_df.columns and 'order_value' in orders_df.columns:
        orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])
        
        fig, axes = plt.subplots(2, 2, figsize=(20, 12))
        fig.suptitle('📅 Time Series Analysis - Orders Dataset', fontsize=16, fontweight='bold')
        
        # 1. Daily revenue trend
        daily_revenue = orders_df.groupby('order_date')['order_value'].sum()
        axes[0, 0].plot(daily_revenue.index, daily_revenue.values, color=AD_COLORS['primary'], linewidth=2)
        axes[0, 0].set_title('Daily Revenue Trend')
        axes[0, 0].set_xlabel('Date')
        axes[0, 0].set_ylabel('Revenue (R)')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        # 2. Weekly pattern
        orders_df['day_of_week'] = orders_df['order_date'].dt.day_name()
        weekly_revenue = orders_df.groupby('day_of_week')['order_value'].sum()
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        weekly_revenue = weekly_revenue.reindex(day_order)
        
        axes[0, 1].bar(weekly_revenue.index, weekly_revenue.values, color=AD_COLORS['secondary'])
        axes[0, 1].set_title('Revenue by Day of Week')
        axes[0, 1].set_xlabel('Day of Week')
        axes[0, 1].set_ylabel('Total Revenue (R)')
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        # 3. Monthly trend
        monthly_revenue = orders_df.groupby(orders_df['order_date'].dt.to_period('M'))['order_value'].sum()
        axes[1, 0].plot(monthly_revenue.index.astype(str), monthly_revenue.values, 
                       marker='o', color=AD_COLORS['accent'], linewidth=3, markersize=8)
        axes[1, 0].set_title('Monthly Revenue Trend')
        axes[1, 0].set_xlabel('Month')
        axes[1, 0].set_ylabel('Revenue (R)')
        axes[1, 0].tick_params(axis='x', rotation=45)
        
        # 4. Rolling average
        daily_revenue_rolling = daily_revenue.rolling(window=7).mean()
        axes[1, 1].plot(daily_revenue.index, daily_revenue.values, alpha=0.3, color=AD_COLORS['neutral'], label='Daily')
        axes[1, 1].plot(daily_revenue_rolling.index, daily_revenue_rolling.values, 
                       color=AD_COLORS['highlight'], linewidth=3, label='7-Day Moving Average')
        axes[1, 1].set_title('Revenue Trend with Moving Average')
        axes[1, 1].set_xlabel('Date')
        axes[1, 1].set_ylabel('Revenue (R)')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.show()

## 9. Key Insights & Recommendations

In [ ]:
# Generate comprehensive insights
print("🎯 KEY INSIGHTS & RECOMMENDATIONS")
print("=" * 50)

insights = []
recommendations = []

# Data Quality Insights
print("
📊 DATA QUALITY INSIGHTS:")
total_records = sum(len(df) for df in datasets.values())
total_missing = sum(df.isnull().sum().sum() for df in datasets.values())
missing_pct = (total_missing / total_records) * 100 if total_records > 0 else 0

print(f"   • Data Completeness: {100-missing_pct:.1f}% ({total_records:,} total records)")
if missing_pct < 1:
    print("   ✅ Excellent data quality - minimal missing values")
    insights.append("High data quality with minimal missing values")
elif missing_pct < 5:
    print("   ⚠️ Good data quality - some missing values to address")
    recommendations.append("Implement data validation rules to prevent missing values")
else:
    print("   ❌ Data quality issues - significant missing values detected")
    recommendations.append("Prioritize data quality improvement initiatives")

# Business Performance Insights
if 'orders' in datasets:
    orders_df = datasets['orders']
    
    print("
💰 BUSINESS PERFORMANCE INSIGHTS:")
    
    if 'order_value' in orders_df.columns:
        total_revenue = orders_df['order_value'].sum()
        avg_order_value = orders_df['order_value'].mean()
        
        print(f"   • Total Revenue: R{total_revenue:,.2f}")
        print(f"   • Average Order Value: R{avg_order_value:,.2f}")
        
        if avg_order_value > 10000:
            print("   💎 High-value customer base identified")
            insights.append("Strong high-value customer segment")
            recommendations.append("Develop premium customer retention programs")
    
    if 'customer_type' in orders_df.columns:
        customer_revenue = orders_df.groupby('customer_type')['order_value'].sum()
        top_segment = customer_revenue.idxmax()
        top_segment_pct = (customer_revenue.max() / customer_revenue.sum()) * 100
        
        print(f"   • Top Revenue Segment: {top_segment} ({top_segment_pct:.1f}%)")
        
        if top_segment_pct > 50:
            print(f"   ⚠️ High dependency on {top_segment} segment")
            recommendations.append(f"Diversify customer base beyond {top_segment} segment")
        else:
            print("   ✅ Well-balanced customer portfolio")
            insights.append("Balanced customer segment distribution")

# Inventory Insights
if 'inventory' in datasets:
    inventory_df = datasets['inventory']
    
    print("
📋 INVENTORY INSIGHTS:")
    
    if 'stock_status' in inventory_df.columns:
        excess_pct = (inventory_df['stock_status'] == 'Excess').mean() * 100
        low_pct = (inventory_df['stock_status'] == 'Low').mean() * 100
        
        print(f"   • Excess Stock: {excess_pct:.1f}% of products")
        print(f"   • Low Stock: {low_pct:.1f}% of products")
        
        if excess_pct > 30:
            print("   ⚠️ High excess inventory levels detected")
            recommendations.append("Implement inventory optimization strategies")
            recommendations.append("Review demand planning accuracy")
        
        if low_pct > 20:
            print("   ⚠️ Risk of stockouts - many products at low levels")
            recommendations.append("Enhance safety stock calculations")
    
    if 'inventory_value' in inventory_df.columns:
        total_inventory_value = inventory_df['inventory_value'].sum()
        print(f"   • Total Inventory Investment: R{total_inventory_value:,.2f}")

# Forecast Performance Insights
if 'forecasts' in datasets:
    forecasts_df = datasets['forecasts']
    
    print("
🎯 FORECAST PERFORMANCE INSIGHTS:")
    
    if 'variance_pct' in forecasts_df.columns:
        forecasts_df['absolute_error'] = abs(forecasts_df['variance_pct'])
        overall_accuracy = 100 - forecasts_df['absolute_error'].mean()
        
        print(f"   • Overall Forecast Accuracy: {overall_accuracy:.1f}%")
        
        if overall_accuracy > 85:
            print("   ✅ Excellent forecast performance")
            insights.append("Strong forecasting capabilities")
        elif overall_accuracy > 75:
            print("   📈 Good forecast performance with room for improvement")
            recommendations.append("Fine-tune forecasting models for better accuracy")
        else:
            print("   ❌ Forecast accuracy needs significant improvement")
            recommendations.append("Overhaul forecasting methodology")
            recommendations.append("Invest in advanced forecasting tools")
    
    if 'forecast_type' in forecasts_df.columns:
        type_performance = forecasts_df.groupby('forecast_type')['absolute_error'].mean()
        best_method = type_performance.idxmin()
        best_accuracy = 100 - type_performance.min()
        
        print(f"   • Best Performing Method: {best_method} ({best_accuracy:.1f}% accuracy)")
        recommendations.append(f"Expand use of {best_method} forecasting approach")

# Summary
print(f"
🎯 STRATEGIC RECOMMENDATIONS:")
for i, rec in enumerate(recommendations[:5], 1):
    print(f"   {i}. {rec}")

print(f"
✅ EDA COMPLETED SUCCESSFULLY!")
print(f"📊 Generated {len(insights)} key insights")
print(f"🎯 Provided {len(recommendations)} strategic recommendations")
print(f"📈 Ready for advanced modeling and dashboard development")

## 10. Export Results

In [ ]:
# Export EDA results
print("💾 EXPORTING EDA RESULTS")
print("=" * 30)

# Create summary report
eda_summary = {
    'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M'),
    'datasets_analyzed': list(datasets.keys()),
    'total_records': sum(len(df) for df in datasets.values()),
    'data_quality_score': 100 - missing_pct,
    'key_insights': insights[:10],  # Top 10 insights
    'recommendations': recommendations[:10]  # Top 10 recommendations
}

# Add dataset-specific metrics
if 'orders' in datasets:
    orders_df = datasets['orders']
    if 'order_value' in orders_df.columns:
        eda_summary['total_revenue'] = orders_df['order_value'].sum()
        eda_summary['avg_order_value'] = orders_df['order_value'].mean()
        eda_summary['total_orders'] = len(orders_df)

if 'inventory' in datasets:
    inventory_df = datasets['inventory']
    if 'inventory_value' in inventory_df.columns:
        eda_summary['total_inventory_value'] = inventory_df['inventory_value'].sum()

if 'forecasts' in datasets:
    forecasts_df = datasets['forecasts']
    if 'variance_pct' in forecasts_df.columns:
        eda_summary['forecast_accuracy'] = 100 - abs(forecasts_df['variance_pct']).mean()

# Save summary to JSON
import json
with open('eda_summary_report.json', 'w') as f:
    json.dump(eda_summary, f, indent=2, default=str)

print("✅ EDA summary exported to 'eda_summary_report.json'")

# Export cleaned datasets for modeling
for name, df in datasets.items():
    filename = f'{name}_eda_processed.csv'
    df.to_csv(filename, index=False)
    print(f"📊 Exported {name} dataset: {filename}")

print(f"
🎉 EDA PROCESS COMPLETED!")
print(f"📈 All insights and data ready for S&OP modeling")
print(f"🔥 Proceed to advanced analytics and dashboard development")